In [1]:
#下载pretrained resnet101
!wget https://download.pytorch.org/models/resnet101-5d3b4d8f.pth -P ./modeling/backbone/

--2020-07-30 17:35:16--  https://download.pytorch.org/models/resnet101-5d3b4d8f.pth
正在解析主机 download.pytorch.org (download.pytorch.org)... 99.84.133.37, 99.84.133.87, 99.84.133.59, ...
正在连接 download.pytorch.org (download.pytorch.org)|99.84.133.37|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 178728960 (170M) [application/octet-stream]
正在保存至: “./modeling/backbone/resnet101-5d3b4d8f.pth.1”

resnet101-5d3b4d8f. 100%[===================>] 170.45M  12.0MB/s    用时 15s     

2020-07-30 17:35:31 (11.6 MB/s) - 已保存 “./modeling/backbone/resnet101-5d3b4d8f.pth.1” [178728960/178728960])



In [2]:
import math
import torch
import torch.nn as nn
from torchsummaryX import summary

In [3]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None, BatchNorm=None):
        super(BasicBlock, self).__init__()
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, 
                                dilation=dilation, padding=dilation, bias=False)
        self.bn1 = BatchNorm(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1, bias=False)
        self.bn2 = BatchNorm(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


In [4]:
class Bottleneck(nn.Module):
    expansion = 4
    
    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None, BatchNorm=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = BatchNorm(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               dilation=dilation, padding=dilation, bias=False)
        self.bn2 = BatchNorm(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = BatchNorm(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.dilation = dilation

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [5]:
class ResNet(nn.Module):

    def __init__(self, block, layers, output_stride, BatchNorm, model_trained, pretrained=True):
        self.inplanes = 64
        super(ResNet, self).__init__()
        blocks = [1, 2, 4]
        if output_stride == 16:
            strides = [1, 2, 2, 1]
            dilations = [1, 1, 1, 2]
        elif output_stride == 8:
            strides = [1, 2, 1, 1]
            dilations = [1, 1, 2, 4]
        else:
            raise NotImplementedError
        
        # Modules
        # 删除头部层
#         self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
#         self.bn1 = BatchNorm(64)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0], stride=strides[0], dilation=dilations[0], BatchNorm=BatchNorm)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=strides[1], dilation=dilations[1], BatchNorm=BatchNorm)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=strides[2], dilation=dilations[2], BatchNorm=BatchNorm)
        self.layer4 = self._make_MG_unit(block, 512, blocks=blocks, stride=strides[3], dilation=dilations[3], BatchNorm=BatchNorm)
        # self.layer4 = self._make_layer(block, 512, layers[3], stride=strides[3], dilation=dilations[3], BatchNorm=BatchNorm)
        self._init_weight()
        
        if pretrained:
            self._load_pretrained_model(model_trained)

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1, BatchNorm=None):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                BatchNorm(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, dilation, downsample, BatchNorm))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation, BatchNorm=BatchNorm))

        return nn.Sequential(*layers)

    def _make_MG_unit(self, block, planes, blocks, stride=1, dilation=1, BatchNorm=None):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                BatchNorm(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, dilation=blocks[0]*dilation,
                            downsample=downsample, BatchNorm=BatchNorm))
        self.inplanes = planes * block.expansion
        for i in range(1, len(blocks)):
            layers.append(block(self.inplanes, planes, stride=1,
                                dilation=blocks[i]*dilation, BatchNorm=BatchNorm))

        return nn.Sequential(*layers)

    def forward(self, x):

#         x = self.conv1(input)
#         x = self.bn1(x)
#         x = self.relu(x)
#         x = self.maxpool(x)
        
        x = self.layer1(x)
        low_level_feat = x
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x, low_level_feat

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            # elif isinstance(m, SynchronizedBatchNorm2d):
            #     m.weight.data.fill_(1)
            #     m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    
    def _load_pretrained_model(self, model_trained):
        # pretrain_dict = model_zoo.load_url('https://download.pytorch.org/models/resnet101-5d3b4d8f.pth')
        pretrain_dict = torch.load(model_trained)
        model_dict = {}
        state_dict = self.state_dict()
        for k, v in pretrain_dict.items():
            if k in state_dict:
                model_dict[k] = v
        state_dict.update(model_dict)
        self.load_state_dict(state_dict)

In [6]:
class ResNet_head(nn.Module):

    def __init__(self):
        super().__init__()
    
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self._init_weight()
        
    def forward(self, x):
 
        x = self.conv1(input)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        return x

    def _init_weight(self):
        for m in self.modules():
#             print("qq",m)
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            # elif isinstance(m, SynchronizedBatchNorm2d):
            #     m.weight.data.fill_(1)
            #     m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

In [7]:
def ResNet101(output_stride, BatchNorm, pretrained=True):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model_trained = 'modeling/backbone/resnet101-5d3b4d8f.pth'
    model = ResNet(Bottleneck, [3, 4, 23, 3], output_stride, BatchNorm, model_trained, pretrained=pretrained)
    return model

def ResNet18(output_stride, BatchNorm, pretrained=True):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model_trained = 'modeling/backbone/resnet18-5c106cde.pth'
    model = ResNet(BasicBlock, [2, 2, 2, 2], output_stride, BatchNorm, model_trained, pretrained=pretrained)
    return model

In [9]:
head = ResNet_head()   #单独把头部部分和下半部分拆分，下半部分读取预训练数据，上半部分自行初始化
model = ResNet101(BatchNorm=nn.BatchNorm2d, pretrained=True, output_stride=8)

input = torch.rand(1, 3, 512, 512)
head_output = head(input)
print("size of head output",head_output.shape)
output, low_level_feat = model(head_output)
print(output.size())
print(low_level_feat.size())

size of head output torch.Size([1, 64, 128, 128])
torch.Size([1, 2048, 64, 64])
torch.Size([1, 256, 128, 128])


In [11]:
summary(model,head_output)

                                             Kernel Shape        Output Shape  \
Layer                                                                           
0_layer1.0.Conv2d_conv1                    [64, 64, 1, 1]   [1, 64, 128, 128]   
1_layer1.0.BatchNorm2d_bn1                           [64]   [1, 64, 128, 128]   
2_layer1.0.ReLU_relu                                    -   [1, 64, 128, 128]   
3_layer1.0.Conv2d_conv2                    [64, 64, 3, 3]   [1, 64, 128, 128]   
4_layer1.0.BatchNorm2d_bn2                           [64]   [1, 64, 128, 128]   
5_layer1.0.ReLU_relu                                    -   [1, 64, 128, 128]   
6_layer1.0.Conv2d_conv3                   [64, 256, 1, 1]  [1, 256, 128, 128]   
7_layer1.0.BatchNorm2d_bn3                          [256]  [1, 256, 128, 128]   
8_layer1.0.downsample.Conv2d_0            [64, 256, 1, 1]  [1, 256, 128, 128]   
9_layer1.0.downsample.BatchNorm2d_1                 [256]  [1, 256, 128, 128]   
10_layer1.0.ReLU_relu       

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_layer1.0.Conv2d_conv1,"[64, 64, 1, 1]","[1, 64, 128, 128]",4096.0,6.710886e+07
1_layer1.0.BatchNorm2d_bn1,[64],"[1, 64, 128, 128]",128.0,6.400000e+01
2_layer1.0.ReLU_relu,-,"[1, 64, 128, 128]",NaN,NaN
3_layer1.0.Conv2d_conv2,"[64, 64, 3, 3]","[1, 64, 128, 128]",36864.0,6.039798e+08
4_layer1.0.BatchNorm2d_bn2,[64],"[1, 64, 128, 128]",128.0,6.400000e+01
...,...,...,...,...
300_layer4.2.BatchNorm2d_bn2,[512],"[1, 512, 64, 64]",1024.0,5.120000e+02
301_layer4.2.ReLU_relu,-,"[1, 512, 64, 64]",NaN,NaN
302_layer4.2.Conv2d_conv3,"[512, 2048, 1, 1]","[1, 2048, 64, 64]",1048576.0,4.294967e+09
